In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Ensure data directory exists
if not os.path.exists('data'):
    os.makedirs('data')

In [ ]:
# Load cleaned datasets
benin_df = pd.read_csv('../data/benin_clean.csv', parse_dates=['Timestamp'])
sierraleone_df = pd.read_csv('../data/sierraleone_clean.csv', parse_dates=['Timestamp'])
togo_df = pd.read_csv('../data/togo_clean.csv', parse_dates=['Timestamp'])

# Add country identifier to each dataframe
benin_df['Country'] = 'Benin'
sierraleone_df['Country'] = 'Sierra Leone'
togo_df['Country'] = 'Togo'

# Combine all data
all_countries = pd.concat([benin_df, sierraleone_df, togo_df], ignore_index=True)

print("Dataset sizes:")
print(f"Benin: {len(benin_df)} rows")
print(f"Sierra Leone: {len(sierraleone_df)} rows")
print(f"Togo: {len(togo_df)} rows")
print(f"Total: {len(all_countries)} rows")

In [ ]:
# Create subplots for GHI, DNI, DHI
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# GHI Boxplot
sns.boxplot(data=all_countries, x='Country', y='GHI', ax=axes[0])
axes[0].set_title('GHI Distribution by Country')
axes[0].set_ylabel('GHI (W/m²)')
axes[0].tick_params(axis='x', rotation=45)

# DNI Boxplot
sns.boxplot(data=all_countries, x='Country', y='DNI', ax=axes[1])
axes[1].set_title('DNI Distribution by Country')
axes[1].set_ylabel('DNI (W/m²)')
axes[1].tick_params(axis='x', rotation=45)

# DHI Boxplot
sns.boxplot(data=all_countries, x='Country', y='DHI', ax=axes[2])
axes[2].set_title('DHI Distribution by Country')
axes[2].set_ylabel('DHI (W/m²)')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate summary statistics for each country
summary_stats = []

for country in ['Benin', 'Sierra Leone', 'Togo']:
    country_data = all_countries[all_countries['Country'] == country]
    
    stats_dict = {
        'Country': country,
        'GHI_Mean': country_data['GHI'].mean(),
        'GHI_Median': country_data['GHI'].median(),
        'GHI_Std': country_data['GHI'].std(),
        'DNI_Mean': country_data['DNI'].mean(),
        'DNI_Median': country_data['DNI'].median(),
        'DNI_Std': country_data['DNI'].std(),
        'DHI_Mean': country_data['DHI'].mean(),
        'DHI_Median': country_data['DHI'].median(),
        'DHI_Std': country_data['DHI'].std(),
        'Sample_Size': len(country_data)
    }
    summary_stats.append(stats_dict)

# Create summary dataframe
summary_df = pd.DataFrame(summary_stats)

# Display formatted table
display_cols = ['Country', 'GHI_Mean', 'GHI_Median', 'GHI_Std', 
                'DNI_Mean', 'DNI_Median', 'DNI_Std',
                'DHI_Mean', 'DHI_Median', 'DHI_Std', 'Sample_Size']

formatted_summary = summary_df[display_cols].round(2)
print("Summary Statistics by Country:")
print(formatted_summary.to_string(index=False))

In [ ]:
# Extract GHI values for each country
benin_ghi = benin_df['GHI'].dropna()
sierraleone_ghi = sierraleone_df['GHI'].dropna()
togo_ghi = togo_df['GHI'].dropna()

print("Statistical Significance Testing for GHI Differences:")

# Test for normality (Shapiro-Wilk)
print("\nNormality Tests (Shapiro-Wilk):")
print(f"Benin: p-value = {stats.shapiro(benin_ghi)[1]:.6f}")
print(f"Sierra Leone: p-value = {stats.shapiro(sierraleone_ghi)[1]:.6f}")
print(f"Togo: p-value = {stats.shapiro(togo_ghi)[1]:.6f}")

# Since solar data often isn't normally distributed, use Kruskal-Wallis test
print("\nKruskal-Wallis Test (non-parametric ANOVA):")
kruskal_stat, kruskal_p = stats.kruskal(benin_ghi, sierraleone_ghi, togo_ghi)
print(f"Kruskal-Wallis H-statistic: {kruskal_stat:.4f}")
print(f"P-value: {kruskal_p:.6f}")

if kruskal_p < 0.05:
    print("Result: Statistically significant differences exist between countries (p < 0.05)")
else:
    print("Result: No statistically significant differences between countries (p >= 0.05)")

# Post-hoc Dunn's test if significant
if kruskal_p < 0.05:
    print("\nPerforming post-hoc analysis...")
    # You can install scikit-posthocs for Dunn's test: pip install scikit-posthocs
    try:
        import scikit_posthocs as sp
        dunn_result = sp.posthoc_dunn([benin_ghi, sierraleone_ghi, togo_ghi], 
                                    p_adjust='bonferroni')
        print("Dunn's post-hoc test results (adjusted p-values):")
        print(dunn_result)
    except ImportError:
        print("Install scikit-posthocs for detailed post-hoc analysis: pip install scikit-posthocs")

In [ ]:
# Create ranking by average GHI
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart ranking by average GHI
ghi_means = all_countries.groupby('Country')['GHI'].mean().sort_values(ascending=False)
colors = ['#2E86AB', '#A23B72', '#F18F01']  # Custom colors

bars = ax1.bar(ghi_means.index, ghi_means.values, color=colors, alpha=0.7)
ax1.set_title('Average GHI by Country\n(Solar Potential Ranking)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Average GHI (W/m²)')
ax1.tick_params(axis='x', rotation=45)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}', ha='center', va='bottom', fontweight='bold')

# Violin plot for distribution comparison
sns.violinplot(data=all_countries, x='Country', y='GHI', ax=ax2, palette=colors)
ax2.set_title('GHI Distribution Comparison', fontsize=14, fontweight='bold')
ax2.set_ylabel('GHI (W/m²)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Technology Recommendation Analysis
print("=== TECHNOLOGY RECOMMENDATION ANALYSIS ===")

# Calculate DNI/DHI ratios for each country
tech_analysis = []

for country in ['Benin', 'Sierra Leone', 'Togo']:
    country_data = all_countries[all_countries['Country'] == country]
    
    # Calculate mean values
    mean_dni = country_data['DNI'].mean()
    mean_dhi = country_data['DHI'].mean()
    mean_ghi = country_data['GHI'].mean()
    mean_ws = country_data['WS'].mean()
    mean_wsgust = country_data['WSgust'].mean()
    
    # Calculate DNI/DHI ratio
    dni_dhi_ratio = mean_dni / mean_dhi if mean_dhi > 0 else float('inf')
    
    tech_analysis.append({
        'Country': country,
        'DNI_Mean': mean_dni,
        'DHI_Mean': mean_dhi,
        'GHI_Mean': mean_ghi,
        'DNI_DHI_Ratio': dni_dhi_ratio,
        'WS_Mean': mean_ws,
        'WSgust_Mean': mean_wsgust
    })

tech_df = pd.DataFrame(tech_analysis).round(3)
print("\nTechnology Analysis Summary:")
print(tech_df[['Country', 'DNI_Mean', 'DHI_Mean', 'DNI_DHI_Ratio', 'WS_Mean']])

# Technology Recommendations
print("\n=== SOLAR TECHNOLOGY RECOMMENDATIONS ===")

# Find country with highest DNI (CSP recommendation)
csp_country = tech_df.loc[tech_df['DNI_Mean'].idxmax(), 'Country']
print(f"• Concentrated Solar Power (CSP): Recommended for {csp_country} "
      f"(Highest DNI: {tech_df['DNI_Mean'].max():.1f} W/m²)")

# Find country with most balanced DNI/DHI ratio (PV recommendation)
tech_df['Ratio_Balance'] = abs(tech_df['DNI_DHI_Ratio'] - 1)  # Distance from ideal balance
pv_country = tech_df.loc[tech_df['Ratio_Balance'].idxmin(), 'Country']
print(f"• Photovoltaic (PV) Systems: Optimal for {pv_country} "
      f"(Most balanced DNI/DHI ratio: {tech_df.loc[tech_df['Ratio_Balance'].idxmin(), 'DNI_DHI_Ratio']:.2f})")

# Find country with best wind patterns (Hybrid recommendation)
wind_country = tech_df.loc[tech_df['WS_Mean'].idxmax(), 'Country']
print(f"• Hybrid Approaches: Beneficial for {wind_country} "
      f"(Highest average wind speed: {tech_df['WS_Mean'].max():.2f} m/s)")

# Visualization for technology analysis
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# DNI vs DHI scatter
colors = {'Benin': 'red', 'Sierra Leone': 'blue', 'Togo': 'green'}
for country in tech_df['Country']:
    country_data = tech_df[tech_df['Country'] == country].iloc[0]
    ax1.scatter(country_data['DHI_Mean'], country_data['DNI_Mean'], 
               s=200, label=country, color=colors[country], alpha=0.7)
ax1.plot([0, max(tech_df['DHI_Mean'])], [0, max(tech_df['DHI_Mean'])], 'k--', alpha=0.3, label='Equal DNI/DHI')
ax1.set_xlabel('DHI (W/m²)')
ax1.set_ylabel('DNI (W/m²)')
ax1.set_title('DNI vs DHI by Country\n(Higher DNI = Better for CSP)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# DNI/DHI Ratio bar chart
bars = ax2.bar(tech_df['Country'], tech_df['DNI_DHI_Ratio'], color=[colors[c] for c in tech_df['Country']])
ax2.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Balanced Ratio (1.0)')
ax2.set_ylabel('DNI/DHI Ratio')
ax2.set_title('DNI to DHI Ratio by Country\n(Ratio ~1.0 = Good for PV)')
ax2.legend()
# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}', ha='center', va='bottom')

# Wind speed comparison
ax3.bar(tech_df['Country'], tech_df['WS_Mean'], color=[colors[c] for c in tech_df['Country']], alpha=0.7)
ax3.set_ylabel('Average Wind Speed (m/s)')
ax3.set_title('Wind Speed by Country\n(Higher = Better for Hybrid)')
# Add value labels
for i, v in enumerate(tech_df['WS_Mean']):
    ax3.text(i, v, f'{v:.2f}', ha='center', va='bottom')

# Technology recommendation summary
tech_recommendations = [
    f"CSP: {csp_country}",
    f"PV: {pv_country}", 
    f"Hybrid: {wind_country}"
]
ax4.axis('off')
ax4.set_title('Recommended Technologies by Country', fontsize=14, fontweight='bold')
for i, rec in enumerate(tech_recommendations):
    ax4.text(0.1, 0.8 - i*0.2, f'• {rec}', fontsize=12, transform=ax4.transAxes)

plt.tight_layout()
plt.show()

# Print detailed technology rationale
print("\n=== TECHNOLOGY RATIONALE ===")
for _, row in tech_df.iterrows():
    country = row['Country']
    dni_ratio = row['DNI_DHI_Ratio']
    ws = row['WS_Mean']
    
    print(f"\n{country}:")
    print(f"  - DNI/DHI Ratio: {dni_ratio:.2f} ({'CSP-favorable' if dni_ratio > 1.2 else 'PV-favorable' if dni_ratio > 0.8 else 'DHI-dominant'})")
    print(f"  - Wind Speed: {ws:.2f} m/s ({'Good for hybrid' if ws > 3.0 else 'Moderate wind' if ws > 2.0 else 'Low wind'})")

## Key Observations

- **Solar Potential Ranking**: Togo shows the highest median GHI of 2.1 W/m², making it the most promising for solar energy generation, while Seirra Leone has the lowest at 0.3 W/m².

- **Variability Analysis**: Benin demonstrates the greatest variability in GHI (std: 331.09), indicating less predictable solar resources compared to Sierra Leone which shows more consistent irradiance patterns.

**Statistical Significance**: The Kruskal-Wallis test reveals **significant differences** between countries (p = 0.000000), suggesting that **the observed variations are likely real geographic differences** rather than random variation.

- **Practical Implications**: Based on average GHI values, solar installations in Benin could potentially generate approximately   240.55% more energy than those in Sierra Leone, highlighting substantial economic implications for solar investment decisions.

In [ ]:
# Monthly patterns comparison
all_countries['Month'] = all_countries['Timestamp'].dt.month

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
for i, country in enumerate(['Benin', 'Sierra Leone', 'Togo']):
    country_monthly = all_countries[all_countries['Country'] == country].groupby('Month')['GHI'].mean()
    axes[i].plot(country_monthly.index, country_monthly.values, marker='o', linewidth=2)
    axes[i].set_title(f'{country} - Monthly GHI Pattern')
    axes[i].set_xlabel('Month')
    axes[i].set_ylabel('Average GHI (W/m²)')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()